# Why do we need padding in CBC?

***By Jinan Jiang, jinan@berkeley.edu***

The following demonstrates that CBC message length must be a multiple of block length, while CFB modes does not have such restriction.

As you can see, CBC only works with messages with length of multiple of the block cipher, while CFB is okay either way.  

The same reasoning applies to other modes of operation that we learned in the class.

In [1]:
from crypto_demo_lib import *

In [2]:
#feel free to change these parameters
num_encs_per_display = 64

## First let's look at CBC

In [9]:
message = b"a 16-byte string" #16 bytes
enctyptable = []
not_enctyptable = []

for _ in range(num_encs_per_display): #different message length in each iteration
    try:
        AES_CBC_enc(data = message, key = random_bytes(32)) #AES-CBC encryption
        enctyptable.append(len(message))
    except:
        not_enctyptable.append(len(message)) #If AES-CBC errored, the message cannot be encrypted
        
    message += b'1' #add 1 byte to the message at each iteration
    
print("CBC can encrypt plaintexts of length: " + str(enctyptable) + '\n')
print("CBC CANNOT encrypt plaintexts of length " + str(not_enctyptable))

CBC can encrypt plaintexts of length: [16, 32, 48, 64]

CBC CANNOT encrypt plaintexts of length [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]


## What about CFB?

In [10]:
message = b"a 16-byte string" #16 bytes
enctyptable = []
not_enctyptable = []

for _ in range(num_encs_per_display):
    try:
        AES_CFB_enc(data = message, key = random_bytes(32)) #AES-CFB encryption
        enctyptable.append(len(message))
    except:
        not_enctyptable.append(len(message)) #If error, the message cannot be encrypted
        
    message += b'1' #add 1 byte to the message at each iteration
    
print("CFB can encrypt plaintexts of length: " + str(enctyptable) + '\n')
print("CFB CANNOT encrypt plaintexts of length " + str(not_enctyptable))

CFB can encrypt plaintexts of length: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]

CFB CANNOT encrypt plaintexts of length []


---

## Why is this the case?

***In a more fundamental level, you can see that plaintexts in the CBC mode must pass through an AES block cipher, which means any input must be a multiple of the size of a block.***

![image info](./images/CBC_encryption.png)

***While for CFB modes, the plaintext is XOR'ed with the output of AES block cipher, which is kind of like an "one time pad". This means we can use as many bits that are output from the block cipher as we need to XOR the plaintext, and discard the rest.***

![image info](./images/CFB_encryption.png)

---

## Now let's try padding plaintexts in CBC

In [11]:
message = b"a 16-byte string" #16 bytes
enctyptable = []
not_enctyptable = []

for _ in range(num_encs_per_display): #different message length in each iteration
    try:
        padded_message = add_padding(message)
        AES_CBC_enc(data = padded_message, key = random_bytes(32)) #AES-CBC encryption
        enctyptable.append(len(message))
    except:
        not_enctyptable.append(len(message)) #If AES-CBC errored, the message cannot be encrypted
        
    message += b'1' #add 1 byte to the message at each iteration
    
print("With padding, CBC can encrypt plaintexts of length: " + str(enctyptable) + '\n')
print("With padding, CBC CANNOT encrypt plaintexts of length " + str(not_enctyptable))

With padding, CBC can encrypt plaintexts of length: [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]

With padding, CBC CANNOT encrypt plaintexts of length []


***With padding, CBC no longer has the restraint on plaintext length. It can encrypt messages of arbitraly length!***

***Feel free to call the functions AES_CBC_dec and remove_padding to verify that this scheme indeed works***

---